# W205 Fall Project 2 - Tracking User Activity

## Nobu Yamaguchi

# Executive summary

> ## The purpose of this analysis is to show the data scientists three basic business questions to answer about these data.

### Question 1: How many assessment are there?

First of all, it is very important to know how many assessments in the dataset? 

### Answer 1. There are 3280 assessments.

Next, in order to understand the overall trend, I answer the following two questions

### Question 2-1: What are the top 10 popular exams?

### Answer 2-1. 
```
+-----------------------------------------------------------+-----+
|exam_name                                             |count|
+-----------------------------------------------------------+-----+
|Learning Git                                                  |394  |
|Introduction to Python                                     |162  |
|Introduction to Java 8                                      |158  |
|Intermediate Python Programming                       |158  |
|Learning to Program with R                                |128  |
|Introduction to Machine Learning                         |119  |
|Software Architecture Fundamentals Understanding the Basics|109  |
|Beginning C# Programming                                |95   |
|Learning Eclipse                                             |85   |
|Learning Apache Maven                                    |80   |
+-----------------------------------------------------------+-----+
```

The table above shows the top 10 popular exams. The most popular one is "Learning Git" with 394 count. Also, there are two python exams in the top 10 list. One is "Introducgtion to Python" and the other is "Intermediate Python Programming".

### Question 2-2: What are the least popular exams (the count is less than 4)?

### Answer 2-2.
```
+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Learning to Visualize Data with D3.js            |1    |
|Native Web Apps for Android                      |1    |
|Nulls, Three-valued Logic and Missing Information|1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
|Learning Spring Programming                      |2    |
|Client-Side Data Storage for Web Developers      |2    |
|Hibernate and JPA Fundamentals                   |2    |
|Arduino Prototyping Techniques                   |2    |
|The Closed World Assumption                      |2    |
|Understanding the Grails 3 Domain Model          |2    |
|What's New in JavaScript                         |2    |
|Service Based Architectures                      |3    |
|Mastering Web Views                              |3    |
|Building Web Services with Java                  |3    |
|Getting Ready for Angular 2                      |3    |
|Using Web Components                             |3    |
+-------------------------------------------------+-----+

```

### Question 3: What are the avarage grades of the 5 top most popular exams?

### Answer 3:
```
+-------------------------------+-------------+---------+
|exam_name                      |average_grade|std_grade|
+-------------------------------+-------------+---------+
|Learning Git                   |68.0         |31.59    |
|Introduction to Python         |57.0         |26.51    |
|Introduction to Java 8         |88.0         |18.78    |
|Intermediate Python Programming|51.0         |29.88    |
|Learning to Program with R     |54.0         |33.07    |
+-------------------------------+-------------+---------+
```

The overall average is 63. However, the top most popular exams are relatively difficult for the users although "Learning Git" and "Introductory to Jave 8" have high average.

In [45]:
import json
import pprint
import sys
from pyspark.sql import Row

In [46]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [47]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [48]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

In [49]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [50]:
extracted_assessments.registerTempTable('assessments')

#### Q1. How many assessment are there?

In [84]:
spark.sql("select count(*) from assessments").show()

+--------+
|count(1)|
+--------+
|    3280|
+--------+



In [86]:
spark.sql("select count(distinct(user_exam_id)) from assessments").show()

+----------------------------+
|count(DISTINCT user_exam_id)|
+----------------------------+
|                        3242|
+----------------------------+



#### Question 2-1: What are the top 10 popular exams?

In [135]:
spark.sql("select exam_name, count(*) as count from assessments group by exam_name order by count(*) desc").show(10, False)

+-----------------------------------------------------------+-----+
|exam_name                                                  |count|
+-----------------------------------------------------------+-----+
|Learning Git                                               |394  |
|Introduction to Python                                     |162  |
|Introduction to Java 8                                     |158  |
|Intermediate Python Programming                            |158  |
|Learning to Program with R                                 |128  |
|Introduction to Machine Learning                           |119  |
|Software Architecture Fundamentals Understanding the Basics|109  |
|Beginning C# Programming                                   |95   |
|Learning Eclipse                                           |85   |
|Learning Apache Maven                                      |80   |
+-----------------------------------------------------------+-----+
only showing top 10 rows



#### Question 2-2: What are the least popular exams (the count is less than 4)?

In [137]:
spark.sql("select exam_name, count(*) as count from assessments group by exam_name having count(*) < 4 order by count").show(20, False)

+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Learning to Visualize Data with D3.js            |1    |
|Native Web Apps for Android                      |1    |
|Nulls, Three-valued Logic and Missing Information|1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
|Learning Spring Programming                      |2    |
|Client-Side Data Storage for Web Developers      |2    |
|Hibernate and JPA Fundamentals                   |2    |
|Arduino Prototyping Techniques                   |2    |
|The Closed World Assumption                      |2    |
|Understanding the Grails 3 Domain Model          |2    |
|What's New in JavaScript                         |2    |
|Service Based Architectures                      |3    |
|Mastering Web Views                              |3    |
|Building Web Services with Java                  |3    |
|Getting Ready

#### Question 3: What are the avarage grades of the 5 top most popular exams?

In [134]:
def my_lambda_grades(x):
    raw_dict = json.loads(x.value)
    my_list = []
    
    for l in raw_dict:
        my_total = 0    
        my_correct = 0
        exam_name = raw_dict["exam_name"]
        if "sequences" in raw_dict:
            if "counts" in raw_dict["sequences"]:
                if "correct" in raw_dict["sequences"]["counts"]:
                    my_correct = raw_dict["sequences"]["counts"]["correct"]
        if "sequences" in raw_dict:
            if "counts" in raw_dict["sequences"]:
                if "total" in raw_dict["sequences"]["counts"]:
                    my_total = raw_dict["sequences"]["counts"]["total"]
        my_dict = {"exam_name" : raw_dict["exam_name"], "correct" : my_correct, "total" : my_total}
        my_list.append(Row(**my_dict))
    return my_list

my_grades = assessments.rdd.flatMap(my_lambda_grades).toDF()
#my_grades.show(50)
#type(my_grades)
#my_grades.count()
my_grades_2 = my_grades.withColumn("grade", my_grades["correct"]/my_grades["total"]*100)
#my_grades_2.show()
my_grades_2.registerTempTable('grades')
#spark.sql("select count(*) from grades").show()
spark.sql("select exam_name, round(avg(grade)) as average_grade, round(stddev_samp(grade),2) as std_grade from grades group by exam_name order by count(exam_name) desc").show(5, False)

+-------------------------------+-------------+---------+
|exam_name                      |average_grade|std_grade|
+-------------------------------+-------------+---------+
|Learning Git                   |68.0         |31.59    |
|Introduction to Python         |57.0         |26.51    |
|Introduction to Java 8         |88.0         |18.78    |
|Intermediate Python Programming|51.0         |29.88    |
|Learning to Program with R     |54.0         |33.07    |
+-------------------------------+-------------+---------+
only showing top 5 rows



In [138]:
spark.sql("select  round(avg(grade)) as average_grade from grades").show()

+-------------+
|average_grade|
+-------------+
|         63.0|
+-------------+



In [133]:
spark.sql("select exam_name, round(avg(grade)) as average_grade, round(stddev_samp(grade),2) as std_grade from grades group by exam_name order by average_grade desc").show(10, False)

+-----------------------------------------------------------------------+-------------+---------+
|exam_name                                                              |average_grade|std_grade|
+-----------------------------------------------------------------------+-------------+---------+
|Learning to Visualize Data with D3.js                                  |100.0        |0.0      |
|Nulls, Three-valued Logic and Missing Information                      |100.0        |0.0      |
|The Closed World Assumption                                            |100.0        |0.0      |
|Learning SQL for Oracle                                                |98.0         |7.22     |
|Introduction to Java 8                                                 |88.0         |18.78    |
|Introduction to Apache Spark                                           |83.0         |16.76    |
|Introduction to Amazon Web Services (AWS) - EC2 Deployment Fundamentals|83.0         |25.67    |
|Getting Ready for A

In [92]:
spark.sql("select exam_name, INT(INT(sequences.counts.correct)/INT(sequences.counts.total)) as grade from assessments group by exam_name ")

AnalysisException: "cannot resolve 'assessments.`sequences`['counts']['correct']' due to data type mismatch: argument 2 requires integral type, however, ''correct'' is of string type.; line 1 pos 26;\n'Aggregate [exam_name#699], [exam_name#699, 'INT(('INT(sequences#704[counts][correct]) / 'INT(sequences#704[counts][total]))) AS grade#1038]\n+- SubqueryAlias assessments\n   +- LogicalRDD [base_exam_id#697, certification#698, exam_name#699, keen_created_at#700, keen_id#701, keen_timestamp#702, max_attempts#703, sequences#704, started_at#705, user_exam_id#706]\n"

In [51]:
spark.sql("select keen_id from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+



In [52]:
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                                  false|
|1516746279.3801291|                                                  false|
| 1516743820.305464|                                                  false|
|  1516743098.56811|                                                  false|
| 1516743764.813107|                                                  false|

In [53]:
spark.sql("select sequences.abc123 from assessments limit 10").show()

+------+
|abc123|
+------+
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
+------+



In [54]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

In [55]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()

In [56]:
my_sequences.registerTempTable('sequences')

In [57]:
spark.sql("select sequences_id from sequences limit 10").show()

+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+



In [58]:
spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844c5-772f-48d...|
|5a51b21bd0480b000...| 1515303451.773272|e7110aed-0d08-4cb...|
|5a575a85329e1a000...| 1515674245.348099|25ca21fe-4dbb-446...|
+--------------------+------------------+--------------------+



In [59]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

In [60]:
my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

In [61]:
my_questions.registerTempTable('questions')

In [62]:
spark.sql("select id, my_count from questions limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+



In [63]:
spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b88-2758-45b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|25b6effe-79b0-4c4...|
|5a26ee9cbf5ce1000...|1512500892.4166169|6de03a9b-2a78-46b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aaf39991-fa83-470...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aab2e817-73dc-4ff...|
+--------------------+------------------+--------------------+



In [24]:
assessments.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [25]:
assessments_as_strings=numbers.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [26]:
assessments_as_strings.show()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
+----+--------------------+
only showing top 20 rows



In [27]:
assessments_as_strings.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [28]:
assessments_as_strings.count()

3280

In [29]:
assessments_as_strings.select('value').take(1)

[Row(value='{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41

In [30]:
assessments_as_strings.select('value').take(1)[0].value

'{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.791Z","id"

In [33]:
first_assessments=json.loads(assessments_as_strings.select('value').take(1)[0].value)

In [34]:
first_assessments

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
  'counts': {'all_correct': False,
   'correct': 2,
   'incomplete': 1,
   'incorrect': 1,
   'submitted': 4,
   'total': 4,
   'unanswered': 0},
  'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
  'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
    'options': [{'at': '2018-01-23T14:23:24.670Z',
      'checked': True,
      'correct': True,
      'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d',
      'submitted': 1},
     {'at': '2018-01-23T14:23:25.914Z',
      'checked': True,
      'correct': True,
      'id': 'f2528210-35c3-4320-acf3-9056567ea19f',
      'submitted': 1},
     {'checked': False,
      'correct': True,
      'id': 'd1bf026f-554f-4543-bdd2-54dcf10

In [35]:
assessments_as_strings.write.parquet("/tmp/assessments")

In [36]:
assessments_as_strings.show()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
+----+--------------------+
only showing top 20 rows



In [38]:
sys.stdout = open(sys.stdout.fileno(), mode='w', encoding='utf8', buffering=1)

UnsupportedOperation: fileno

In [40]:
assessments_as_strings.rdd.map(lambda x: json.loads(x.value)).toDF().show()

/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [41]:
extracted_assessmenrts = assessments_as_strings.rdd.map(lambda x: json.loads(x.value)).toDF()

/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [42]:
extracted_assessmenrts = assessments_as_strings.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [44]:
extracted_assessmenrts.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [2]:
p = pprint.PrettyPrinter(indent=1)

In [3]:
f = open("assessment-attempts-20180128-121051-nested.json","r")

In [4]:
s = f.read()

In [5]:
json_data = json.loads(s)

In [6]:
f.close()

In [7]:
len(json_data)

3280

In [8]:
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [9]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)

In [10]:
recursive_walk_json_object(json_data[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

In [11]:
!cat assessment-attempts-20180128-121051-nested.json | jq ' .[] | .exam_name' -c | sort | uniq -c | sort -r

    394 "Learning Git"
    162 "Introduction to Python"
    158 "Introduction to Java 8"
    158 "Intermediate Python Programming"
    128 "Learning to Program with R"
    119 "Introduction to Machine Learning"
    109 "Software Architecture Fundamentals Understanding the Basics"
     95 "Beginning C# Programming"
     85 "Learning Eclipse"
     80 "Learning Apache Maven"
     79 "Beginning Programming with JavaScript"
     77 "Mastering Git"
     75 "Introduction to Big Data"
     67 "Advanced Machine Learning"
     59 "Learning Linux System Administration"
     58 "JavaScript: The Good Parts Master Class with Douglas Crockford"
     57 "Learning SQL"
     53 "Practical Java Programming"
     52 "HTML5 The Basics"
     51 "Python Epiphanies"
     48 "Software Architecture Fundamentals Beyond The Basics"
     43 "Introduction to Data Science with R"
     43 "Intermediate C# Programming"
     40 "Learning DNS"
     35 "Learning C# Best Practices"
     35 "Expert Data Wrangling with R"
 

In [2]:
messages = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessment").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 


In [3]:
messages.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
messages.show()

+----+--------------------+----------+---------+------+--------------------+-------------+
| key|               value|     topic|partition|offset|           timestamp|timestampType|
+----+--------------------+----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessment|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessment|        0|     7|1969-12-31 23:59:...|            0|

In [5]:
messages_as_strings=messages.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
messages_as_strings.show()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
|null|{"keen_timestamp"...|
+----+--------------------+
only showing top 20 rows



In [7]:
messages_as_strings.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [8]:
messages_as_strings.count()

3280

In [9]:
messages_as_strings.select('value').take(1)

[Row(value='{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41

In [10]:
messages_as_strings.select('value').take(1)[0].value

'{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.791Z","id"

In [11]:
first_message=json.loads(messages_as_strings.select('value').take(1)[0].value)

In [12]:
first_message

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
  'counts': {'all_correct': False,
   'correct': 2,
   'incomplete': 1,
   'incorrect': 1,
   'submitted': 4,
   'total': 4,
   'unanswered': 0},
  'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
  'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
    'options': [{'at': '2018-01-23T14:23:24.670Z',
      'checked': True,
      'correct': True,
      'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d',
      'submitted': 1},
     {'at': '2018-01-23T14:23:25.914Z',
      'checked': True,
      'correct': True,
      'id': 'f2528210-35c3-4320-acf3-9056567ea19f',
      'submitted': 1},
     {'checked': False,
      'correct': True,
      'id': 'd1bf026f-554f-4543-bdd2-54dcf10